# Strands マルチエージェントシステムの AgentCore Memory Tool (短期記憶)

日本語訳:

Strands は、 AgentCore Memory Tool (短期記憶) を備えたマルチエージェントシステムです。 AgentCore Memory Tool は、エージェントの短期記憶を管理するツールです。このツールを使用することで、エージェントは一時的な情報を記憶し、後で参照することができます。

## 序文

この Notebook では、AWS AgentCore Memory と Strands フレームワークを使用して **共有メモリを持つマルチエージェントシステム** を実装する方法を示します。以前の例では単一エージェントのメモリに焦点を当てていましたが、この Notebook では、複数の専門エージェントが共通のメモリストアにアクセスしながら協力する方法を探ります。

## チュートリアルの詳細

| 情報                | 詳細                                                                              |
|:--------------------|:---------------------------------------------------------------------------------|
| チュートリアルの種類 | 短期会話型                                                                       |
| エージェントのユースケース | 旅行プランアシスタント                                                           |
| エージェントフレームワーク | Strands Agents                                                                   |
| LLM モデル          | Anthropic Claude Sonnet 3                                                        |
| チュートリアルコンポーネント | AgentCore 短期メモリ、Strands Agents、ツールを介したメモリ取得                     |
| 例の複雑さ           | 初級                                                                             |


学習内容:

- 複数のエージェントがアクセスできる共有メモリリソースの設定方法
- 独自のメモリアクセスを持つ専門エージェントをツールとして作成する方法
- 専門エージェントに委任するコーディネーターエージェントの実装方法
- 複数のエージェント対話にわたる会話コンテキストの維持

### シナリオコンテキスト

この例では、以下のような **旅行プランシステム** を作成します。
1. 航空旅行に特化した Flight Booking Assistant
2. 宿泊施設に特化した Hotel Booking Assistant
3. これらの専門エージェントに委任する Travel Coordinator

このアプローチでは、複雑なドメインを同じメモリストアを共有する専門エージェントに分割する方法を示しています。

## アーキテクチャ
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## 前提条件
- Python 3.10 以上
- 適切な権限を持つ AWS アカウント
- AgentCore Memory に対して適切な権限を持つ AWS IAM ロール
- Amazon Bedrock モデルへのアクセス

それでは、環境をセットアップし、共有メモリリソースを作成しましょう!

## ステップ 1: 環境設定
必要なすべてのライブラリをインポートし、このノートブックを動作させるためのクライアントを定義しましょう。

In [ ]:
!pip install -qr requirements.txt

In [ ]:
import logging
from datetime import datetime
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent

以下が日本語訳になります。

Amazon Bedrock models と AgentCore に適切な権限を持つリージョンとロールを定義してください。

In [ ]:
import os
region = os.getenv('AWS_REGION', 'us-west-2')

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("agentcore-memory")

## ステップ 2: 共有メモリの作成
このセクションでは、特化したエージェント間で共有されるメモリリソースを作成します。

In [ ]:
from bedrock_agentcore.memory import MemoryClient

In [ ]:
client = MemoryClient(region_name=region)
memory_name = "TravelAgent_STM_%s" % datetime.now().strftime("%Y%m%d%H%M%S")
memory_id = None

In [ ]:
from botocore.exceptions import ClientError

try:
    print("Creating Memory...")
    memory_name = memory_name

    # Create the memory resource

メモリリソースを作成します。

resource "aws_memorydb_cluster" "example" {
  cluster_name         = "example-memorydb-cluster"
  node_type            = "db.r6g.large"
  num_shards           = 2
  num_replicas_per_shard = 1
  acl_name             = aws_memorydb_acl.example.name
  subnet_group_name    = aws_memorydb_subnet_group.example.name
  security_group_ids   = [aws_security_group.example.id]
  snapshot_retention_limit = 5
  maintenance_window   = "sun:05:00-sun:09:00"

  parameter_group_name = aws_memorydb_parameter_group.example.name

  tags = {
    Environment = "production"
  }
}

resource "aws_memorydb_parameter_group" "example" {
  name   = "example-memorydb-params"
  family = "memorydb_redis6"
}

resource "aws_memorydb_subnet_group" "example" {
  name       = "example-memorydb-subnet-group"
  subnet_ids = [aws_subnet.example_1.id, aws_subnet.example_2.id]
}

resource "aws_memorydb_acl" "example" {
  name        = "example-memorydb-acl"
  user_names  = ["default"]
  minimum_engine_version = "6.2"
}
    memory = client.create_memory_and_wait(
        name=memory_name,                       # Unique name for this memory store

このメモリストアの一意の名前
        description="Travel Agent STM",         # Human-readable description

人間が読めるように説明します。

半角英数字の前後には半角スペースを挿入します。 コード や コマンド 、 変数名 、 関数名 などの技術的な用語は翻訳せず、そのまま残します。
        strategies=[],                          # 短期記憶のための特別な記憶戦略はありません

日本語訳:

短期記憶のための特別な記憶戦略はありません。短期記憶は、一時的に情報を保持する機能です。この機能は、 working_memory と呼ばれることもあります。短期記憶の容量は限られているため、長期記憶に情報を転送する必要があります。

長期記憶に情報を転送するための一般的な戦略には、次のようなものがあります。

- 繰り返し (rehearsal)
- 関連付け (association)
- 視覚化 (visualization)
- 手がかり (cues)

これらの戦略は、長期記憶への情報の転送を促進しますが、短期記憶自体の容量を拡張するものではありません。短期記憶の容量は制限されているため、効率的に使用する必要があります。
        event_expiry_days=7,                    # Memories expire after 7 days

日本語訳:
# 7 日後にメモリが期限切れになります
        max_wait=300,                           # 最大待機時間 (5 分) メモリ作成を待つ

日本語訳:
# メモリ作成を待つ最大時間 (5 分)
        poll_interval=10                        # 10秒ごとにステータスを確認する

日本語訳:
10 秒ごとに status を確認します。 check、every、seconds は技術的な用語なので翻訳しませんでした。
    )

    # メモリIDを抽出して出力する

memory_id = subprocess.check_output([ 'cat', '/proc/meminfo' ])
print( "Memory ID: " + memory_id.split()[1] )
    memory_id = memory['memoryId']
    print(f"Memory created successfully with ID: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID

メモリがすでに存在する場合は、その ID を取得します。
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # メモリ作成中のエラーを処理する

try :
    memory = create_memory ( )
except MemoryError as error :
    print ( f"Error creating memory: { error }" )
    sys.exit ( 1 )

try :
    data = load_data ( )
    memory.load ( data )
except ( DataError , MemoryError ) as error :
    print ( f"Error loading data into memory: { error }" )
    sys.exit ( 1 )

print ( "Memory loaded successfully." )
    print(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()

    # Unique name for this memory store

このメモリストアの一意の名前0
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

### マルチエージェントシステムにおける共有メモリの理解

私たちが作成したメモリリソースは、旅行計画システムの共有知識ベースとして機能します。すべてのエージェントがこの共通のメモリストアから読み取り、書き込みを行うことで、以下が可能になります。

1. **知識の一貫性**: すべてのエージェントが同じ情報を共有
2. **コンテキストの保存**: 会話履歴がエージェントの切り替えを通して維持される
3. **特化されたアクセス**: 各エージェントは独自の actor_id を持ちながら、session_id を共有

このアプローチにより、特化したエージェントはそれぞれの分野に専念しながら、完全な会話コンテキストの恩恵を受けることができます。

## ステップ 3: Memory Hook Provider の作成

このステップでは、メモリ操作を自動化するカスタム `MemoryHookProvider` クラスを定義します。フックとは、エージェントの実行ライフサイクルの特定の時点で実行される特別な関数です。作成するメモリフックには、主に次の 2 つの機能があります。

1. **Memories の取得**: ユーザーがメッセージを送信したときに、関連する過去の会話を自動的に取得する
2. **Memories の保存**: エージェントが応答した後に、新しい会話を保存する

これにより、手動での管理なしにシームレスなメモリ体験が実現します。

In [ ]:
class ShortTermMemoryHook(HookProvider):
    def __init__(self, memory_client: MemoryClient, memory_id: str, actor_id: str, session_id: str):
        self.memory_client = memory_client
        self.memory_id = memory_id
        self.actor_id = actor_id
        self.session_id = session_id
    
    def on_agent_initialized(self, event: AgentInitializedEvent):
        翻訳するテキスト:
"""Load recent conversation history when agent starts"""

日本語訳:
"""エージェントが開始するときに 最近の会話履歴を読み込む"""
        try:
            # Get last 5 conversation turns

最後の 5 つの会話ターンを取得します。

conversation_history = conversation.history[-5:]

for turn in conversation_history:
    print(f"Human: {turn.human_utterance}")
    print(f"Assistant: {turn.ai_utterance}\n")
            recent_turns = self.memory_client.get_last_k_turns(
                memory_id=self.memory_id,
                actor_id=self.actor_id,
                session_id=self.session_id,
                k=5,
                branch_name="main"
            )
            
            if recent_turns:
                # 会話履歴をコンテキストとしてフォーマットする

日本語訳:

# フォーマット化された会話履歴

context = ""
for i, (query, response) in enumerate(conversation_history):
    context += f" {i+1} 人間: {query} \n アシスタント: {response}"
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        role = message['role'].lower()
                        content = message['content']['text']
                        context_messages.append(f"{role.title()}: {content}")
                
                context = "\n".join(context_messages)
                logger.info(f"Context from memory: {context}")
                
                # Add context to agent's system prompt

システムプロンプトにコンテキストを追加する
                event.agent.system_prompt += f"\n\nRecent conversation history:\n{context}\n\nContinue the conversation naturally based on this context."
                
                logger.info(f"✅ Loaded {len(recent_turns)} recent conversation turns")
            else:
                logger.info("No previous conversation history found")
                
        except Exception as e:
            logger.error(f"Failed to load conversation history: {e}")
    
    def on_message_added(self, event: MessageAddedEvent):
        以下が日本語訳になります。

"""会話の発話を memory に保存する"""

技術的な用語は翻訳せずにそのまま残しました。半角英数字の前後には半角スペースを挿入しています。
        messages = event.agent.messages
        try:
            self.memory_client.create_event(
                memory_id=self.memory_id,
                actor_id=self.actor_id,
                session_id=self.session_id,
                messages=[(messages[-1]["content"][0]["text"], messages[-1]["role"])]
            )
            
        except Exception as e:
            logger.error(f"Failed to store message: {e}")
    
    def register_hooks(self, registry: HookRegistry) -> None:
        # メモリフックの登録

日本語訳:

この関数は、指定されたアドレス範囲に対してメモリ監視フックを登録します。 `VirtualProtectEx` を使用して、指定された範囲のメモリ保護属性を一時的に変更し、 `PAGE_EXECUTE_READWRITE` フラグを設定します。次に、 `MapViewOfFileEx` を使用して、指定された範囲のメモリ領域にマップされたビューを作成します。作成されたビューは、元のメモリ領域の内容を反映しています。

その後、 `GetProcAddress` を使用して、 `ntdll.dll` から `NtProtectVirtualMemory` 関数のアドレスを取得します。この関数は、メモリ保護属性を変更するためのネイティブ Windows API 関数です。

最後に、 `NtProtectVirtualMemory` を呼び出して、指定された範囲のメモリ保護属性を `PAGE_EXECUTE_READ` に変更します。これにより、指定された範囲のメモリが実行可能になり、読み取り専用になります。

この関数は、指定された範囲のメモリ領域を監視するために使用されます。メモリが変更されると、フックされた関数が呼び出されます。
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)

## ステップ 4: Strands Agents を使ってマルチエージェントアーキテクチャを作成する
このセクションでは、フライトと宿泊施設の予約を専門とするエージェントが memory リソースへのアクセスを共有するマルチエージェントシステムを作成します。

In [ ]:
# Import the necessary components

必要なコンポーネントを import します。
from strands import Agent, tool

In [ ]:
# 各特化エージェントに対して一意のアクターIDを作成するが、セッションIDは共有する

日本語訳:

各特化エージェントに対して 一意の actor_id を作成しますが、session_id は共有します。
flight_actor_id = f"flight-user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
hotel_actor_id = f"hotel-user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
session_id = f"travel-session-{datetime.now().strftime('%Y%m%d%H%M%S')}"
flight_namespace = f"travel/{flight_actor_id}/preferences"
hotel_namespace = f"travel/{hotel_actor_id}/preferences"

### 記憶アクセス機能を持つ特化エージェントの作成

次に、特化エージェントのシステムプロンプトを定義します。各プロンプトには、エージェントが解析できる形式のメモリパラメータが含まれています。

日本語訳:

記憶アクセス機能を持つ特化エージェントの作成

次に、私たちは特化エージェントのシステムプロンプトを定義します。各プロンプトには、エージェントが解析できる形式の memory パラメータが含まれています。

In [ ]:
# System prompt for the hotel booking specialist

# ホテル予約専門家のためのシステムプロンプト
HOTEL_BOOKING_PROMPT = f"""あなたはホテル予約アシスタントです。顧客がホテルを見つけ、予約を行い、宿泊施設やアメニティについての質問に答えられるよう支援します。
空室状況、料金、予約手順について、フレンドリーで親切な態度で明確な情報を提供してください。"""

# System prompt for the flight booking specialist

# フライト予約専門家のためのシステムプロンプト
FLIGHT_BOOKING_PROMPT = f"""あなたは航空券予約アシスタントです。顧客が航空便を探し、予約を行い、航空会社、路線、旅行規定に関する質問に答えられるよう支援します。
フライトの空席状況、価格、スケジュール、予約手順について、フレンドリーで親切な態度で明確な情報を提供してください。"""

### エージェントツールの実装
次に、コーディネーターエージェントが使用できる専門エージェントをツールとして実装します。

日本語訳:
コーディネーターエージェントが使用できる専門的なエージェントをツールとして実装します。 

コード、コマンド、変数名、関数名などの技術的な用語は翻訳せず、そのままの形で残します。半角英数字の前後には半角スペースを挿入します。

In [ ]:
@tool
def flight_booking_assistant(query: str) -> str:
    """
    フライト予約に関する問い合わせを処理し、回答します。

    Args:
        query: 予約、スケジュール、航空会社、旅行規定に関する航空機関連の質問

    Returns:
        詳細な航空情報、予約オプション、または旅行に関するアドバイス
"""
    try:
        flight_memory_hooks = ShortTermMemoryHook(
            memory_id=memory_id,
            memory_client=client,
            actor_id=flight_actor_id,
            session_id=session_id
        )
        
        flight_agent = Agent(hooks=[flight_memory_hooks], system_prompt=FLIGHT_BOOKING_PROMPT)

        # Call the agent and return its response

エージェントを呼び出し、その応答を返します。

response = agent.run( input )
        response = flight_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in flight booking assistant: {str(e)}"

@tool
def hotel_booking_assistant(query: str) -> str:
    """
    ホテルの予約に関する問い合わせを処理し、回答します。

    Args:
        query: 宿泊施設、アメニティ、予約に関するホテル関連の質問

    Returns:
        詳細なホテル情報、予約オプション、または宿泊に関するアドバイス
"""
    try:
        hotel_memory_hooks = ShortTermMemoryHook(
            memory_id=memory_id,
            memory_client=client,
            actor_id=hotel_actor_id,
            session_id=session_id
        )

        hotel_booking_agent = Agent(hooks=[hotel_memory_hooks], system_prompt=HOTEL_BOOKING_PROMPT)
        
        # Call the agent and return its response

エージェントを呼び出し、その応答を返します。

response = agent.run( input )
        response = hotel_booking_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in hotel booking assistant: {str(e)}"

### 調整役エージェントの作成

最後に、これらの専門ツールを調整する主要な旅行計画エージェントを作成します。

日本語訳:
### 調整役エージェントの作成

最後に、これらの専門ツールを調整する主要な旅行計画 agent を作成します。

import json
import os

import openai
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

def get_travel_plan(destination, budget, duration):
    prompt = f"""
You are a travel agent assistant. Your task is to create a travel plan for a trip to {destination} with a budget of {budget} and duration of {duration} days.

The travel plan should include:
1. Flight details (departure and return dates, airlines, etc.)
2. Accommodation details (hotel names, locations, nightly rates, etc.)
3. A daily itinerary of activities and attractions to visit
4. Any other relevant information (travel advisories, visa requirements, etc.)

Please provide the travel plan in a structured format, such as a JSON object or a Markdown table.
"""

    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=2048,
        n=1,
        stop=None,
        temperature=0.5,
    )

    travel_plan = response.choices[0].text.strip()
    return travel_plan

# Example usage
destination = "Tokyo"
budget = 5000
duration = 7

travel_plan = get_travel_plan(destination, budget, duration)
print(travel_plan)

In [ ]:
# System prompt for the coordinator agent

システムプロンプトのコーディネーター agent です。

以下のように翻訳しました。

- 半角英数字の前後に半角スペースを挿入しました。
- コードやコマンド、変数名、関数名などの技術的な用語は翻訳せず、そのまま残しました。

技術的な用語が含まれていないシンプルなテキストでしたので、そのまま直訳しました。必要に応じて、より分かりやすい言い回しに修正することをお勧めします。
TRAVEL_AGENT_SYSTEM_PROMPT = あなたは包括的な旅行プランニングアシスタントで、次の専門ツールを連携させます。

- 航空便に関する問い合わせ(予約、スケジュール、航空会社、路線)の場合 → flight_booking_assistant ツールを使用
- ホテルに関する問い合わせ(宿泊施設、アメニティ、予約)の場合 → hotel_booking_assistant ツールを使用 
- 完全な旅行パッケージの場合 → 必要に応じて両方のツールを使用し、包括的な情報を提供
- 一般的な旅行アドバイスや簡単な旅行の質問の場合 → 直接回答

各エージェントには、ユーザーが過去のデータについて尋ねた場合に備えて、独自のメモリがあります。
複雑な旅行リクエストを処理する際は、両方のツールから情報を調整し、まとまった旅行プランを作成します。
複数のソースから情報を提示する際は、明確な構成を心がけてください。
1回のターンで質問は最大2つまでにしてください。メッセージは短く、顧客を圧倒しないようにしてください。

In [ ]:
travel_agent = Agent(
    system_prompt=TRAVEL_AGENT_SYSTEM_PROMPT,
    tools=[flight_booking_assistant, hotel_booking_assistant]
)

#### あなたのマルチエージェントシステムの準備ができました!!

## エージェントをテストしましょう

旅行計画のシナリオでマルチエージェントシステムをテストしましょう:

In [ ]:
response = travel_agent("Hello, I would like to book a trip from LA to Madrid. From July 1 to August 2.")

In [ ]:
response = travel_agent("I would only like to focus on the flight at the moment. direct flimid-range, city center, pool, standard room")

## メモリの永続性のテスト

メモリシステムが正しく機能しているかどうかをテストするために、旅行代理店の新しいインスタンスを作成し、以前に保存された情報にアクセスできるかどうかを確認します。

In [ ]:
# 新しい travel agent のインスタンスを作成する

日本語訳:
# 新しい旅行代理店のインスタンスを作成します
new_travel_agent = Agent(
    system_prompt=TRAVEL_AGENT_SYSTEM_PROMPT,
    tools=[flight_booking_assistant, hotel_booking_assistant]
)

# Ask about previous conversations

以前の会話について尋ねる

user_input = input("Would you like to see previous conversations? (y/n) ")
if user_input.lower() == "y":
    # 過去の会話を表示する処理
    print("Showing previous conversations...")
else:
    print("OK, no previous conversations will be shown.")
new_travel_agent("Can you remind me about flights talked about before?")

## 概要

このノートブックでは、以下のことを実演しました。

1. 複数の エージェント 用の共有メモリリソースを作成する方法
2. メモリアクセス機能を持つ特殊な エージェント をツールとして実装する方法
3. 会話コンテキストを維持しながら、複数の エージェント 間で調整する方法
4. メモリが異なる エージェント インスタンス間で永続化する方法

共有メモリを備えたこのマルチ エージェント アーキテクチャは、特化したドメインを扱えるとともに、一貫したユーザー体験を維持できる複雑な会話 AI システムを構築するための強力なアプローチを提供します。

## クリーンアップ
このノートブックで使用したリソースを解放するため、メモリを削除しましょう。

In [ ]:
#client.delete_memory_and_wait(

日本語訳:
# client.delete_memory_and_wait( は、技術的な用語のようですので、そのまま残します。

この行は、おそらく何らかのプログラミング言語のコードの一部で、クライアントオブジェクトの delete_memory_and_wait メソッドを呼び出しているものと思われます。この関数は、おそらくメモリ上のデータを削除し、その処理が完了するまで待機する役割を持っているのでしょう。
#        memory_id = memory_id,

日本語訳:

#        memory_id = memory_id,
#        max_wait = 300,

日本語訳:
#        max_wait = 300,
#        poll_interval =10

日本語訳:

#        poll_interval = 10
翻訳するテキスト:

# )

日本語訳:

# ) は空のコメントです。